In [5]:
! pip install -qqq datasets sacrebleu torch transformers 
! pip install -qqq sentencepiece transformers[sentencepiece]
! pip install -qqq accelerate -U
! pip install -qqq scikit-learn datasets peft evaluate
! pip install -U bitsandbytes


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from peft import PeftModel, PeftConfig
import torch

model_name = "IrbisAI/Irbis-7b-Instruct_lora"
config = PeftConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto")
model = PeftModel.from_pretrained(model, model_name)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Generating...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<s> Сен — қазақ тілінде сөйлейтін автоматты көмекші Далбаеб. Төменде тапсырма және қосымша контекст беретін енгізу келтірілген. Дұрыс жауап жаз.

### Тапсырма:
Сәлем, сенің атың кім?

### Енгізу:


### Жауап:
Менің атым - Джейн</s>


In [6]:
import time
start = time.time()
question = "Менің атым Ануар, жасым 20-да, SimpleCode компаниясында программист болып жұмыс істеймін, бос уақытымда шахмат ойнағанды ​​ұнатамын. Мен коммуникативтімін."
context  = ""


template = """Сен — қазақ тілінде сөйлейтін автоматты көмекші. Төменде тапсырма және қосымша контекст беретін енгізу келтірілген. Дұрыс жауап жаз.
Осы мәтіннен түйінді сөздерді шығарып алыңыз, атап айтқанда: Аты, Жасы, Біліктілігі, Мамандығы. Жауабыңыз JSON пішімінде болуы керек. JSON пішімі келесідей болуы керек: {"name":str, "age":int, "skill":str, "job":str}
### Тапсырма:



### Жауап:
"""

input_ids = tokenizer([template.format(question=question, context=context)], return_tensors = "pt")["input_ids"].to("cuda")

generation_config = GenerationConfig(
    temperature=0.6,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=2048,
    pad_token_id=tokenizer.eos_token_id,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s)) # Жасыл шөптің түсі өсімдіктегі хлорофилл деп аталатын химиялық затқа байланысты. Хлорофилл күн сәулесін сіңіреді, содан кейін оны жасушаларға жібереді. Бұл жасушалар жарық энергиясын көмірқышқыл газын оттегімен тотықтырады, бұл процесс арқылы энергия өндіріледі.

end = time.time()

print(f'execution: {end - start}')

KeyError: '"name"'